<a href="https://colab.research.google.com/github/martatolos/eae-dsaa-2025/blob/main/nlp_prompting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Prompt Engineering

> Goal of the session:
>
> - At the end of this activity, you will be able to write your own prompts efficiently and strategies to build them.
>
>Scope of the session
>
> - We will walk through the basic concepts to learn the basics on prompt engineering.
> - We will use an instruction tuned OpenAI LLM
> - How to write clear and specific prompts
>   - Use of delimiters
>   - Structured output
>   - Conditions check
>   - Few-shot prompting
> - How to let the model simulate thinking process

###  1. Base LLMs vs Instruction Tuned LLMs

First, let's understand the difference between a base LLM and an instruction-tuned LLM.

#### Base LLM

Predicts next word, based on text training data.

Examples:

Prompt
```
Once upon a time, there was a unicorn
```

Answer
```
that lived in a magical forest with all her unicorn friends
```

Prompt
```
What is the capital of France?
```

Answer
```
What is France's largest city?
What is France's popilation?
What is the currency of France?
```

#### Instruction Tuned LLM

Tries to follow instructions, as it is fine-tuned on instructions and good examples at following those instructions.

Examples:

Prompt
```
What is the capital of France?
```

Answer
```
The capital of France is Paris.
```

### 2. Prompt Engineering Basic Understandings

#### What is Prompt Engineering

Prompt engineering refers to the process of crafting effective prompts or input queries to elicit desired responses from the model. This process involves understanding the capabilities and limitations of the LLM and designing prompts that guide it towards generating the desired outputs.

#### What is a prompt

In the context of Generative AI, a prompt is a specific input provided to an AI model to generate desired outputs, such as text, images, or even music. The prompt serves as the starting point or guiding instruction for the model, influencing the content and style of the generated output.

Example of prompt:
```
Could you recommend 5 lesser-known spots in Barcelona?
```

### Prompt engineering (some) techniques

Prompt engineering can include various techniques such as:

1. **Prompt formulation**: Crafting the wording and structure of the prompt to provide the necessary context and constraints for the model to generate the desired response.
2. **Prompt tuning**: Iteratively adjusting the prompt based on the model's responses to improve the quality and relevance of the generated outputs.
3. **Prompt augmentation**: Adding additional information or cues to the prompt to guide the model towards specific types of responses or to encourage more diverse outputs.
4. **Prompt analysis**: Analyzing the effectiveness of different prompts in influencing the model's behavior and using this insight to refine future prompt designs.

### 3. Setup

#### Dependencies

- ``ipython``
- ``openai`` 1.75.0
- ``python-dotenv``

In [ ]:
%pip install ipython openai==1.75.0 python-dotenv

### Imports

In [ ]:
import json
import os

import dotenv
from IPython.display import Markdown, display
from openai import OpenAI

### API Key

Add your OpenAI API key in the cell below or create a `.env` file in the same directory as this notebook with the following content:

```
OPENAI_API_KEY=your_openai_api_key
```

> [!Warning]
> Make sure you do not save or commit the file without removing your API key. If that happens, reset the key so that it is not compromised.

In [ ]:
open_ai_key = None  # Add your OpenAI API key here
dotenv.load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY", open_ai_key)

### Helper functions

Next, we will create a function that we will use in today's activity. We will use [chat completions endpoint](https://platform.openai.com/docs/guides/text-generation/chat-completions-api). Also most LLMs use markdown format to in their output. We will use ``render_markdown`` function to show the outputs we get in a more readable format.

In [ ]:
def get_completion(prompt: str, model_name: str = "gpt-4.1-nano") -> str:
    """Get the completion from OpenAI API.

    :param prompt: Prompt to be sent to the model.
    :param model_name: Name of the model which will be used.
        Check https://platform.openai.com/docs/models to get an updated list.
        Defaults to "gpt-4o-mini"
    :return: Completion from the model.
    """
    return OpenAI().responses.create(model=model_name, input=prompt).output[0].content[0].text


def render_markdown(text: str) -> None:
    """Render the text as markdown.

    :param text: Text to be rendered.
    """
    display(Markdown(text))


def show_completion(prompt: str, model_name: str = "gpt-4.1-nano") -> None:
    """Get the completion from OpenAI API and render it as markdown.

    :param prompt: Prompt to be sent to the model.
    :param model_name: Name of the model which will be used.
        Check https://platform.openai.com/docs/models to get an updated list.
        Defaults to "gpt-4o-mini"
    """
    completion = get_completion(prompt, model_name)
    render_markdown(completion)

In [ ]:
prompt = "Tell me a joke"
show_completion(prompt)

## 4. Prompting Basics

### How to write clear and specific prompts

- If our prompt is vague, we are giving freedom to the model to:
  - Decide whether there are any information gaps in our instruction.
  - Fill the information gaps in our instruction, if the model detects that the information in the prompt is not complete.
- Easily, this scenario can lead to poor generated answers or answers with  non-factual content.
- To avoid or mitigate ambiguity in the instruction, our prompt needs to be clear and specific to help the model to focus on a precise task to solve.
- Clarity and specificity does not have anything to do with the length of the prompt (i.e. short prompts do not necessary are clear and specific).

Example of a vague rather short prompt:

In [ ]:
prompt = "Write a blog post about Artificial Intelligence"
show_completion(prompt)

Example of a clear and specific prompt

In [ ]:
prompt = """
Write a 200 words long blog post about Artificial Intelligence. The post should \
include the three most relevant and recent advancements in Artificial Intelligence \
using text to image model techniques. The reader is a general audience.
"""

show_completion(prompt)

### Prompting Techniques

There are some techniques for writing clear and specific prompts. We will explore the following ones:

1. Use delimiters to mark the boundaries of data sources
2. Instruct the model to generate a structured output
3. Check whether the conditions are met
4. Insert examples to show the model to solve the task (few-shot prompting)


#### Use delimiters to mark the boundaries of data sources/task

Examples
- Double quotes: ""
- Triple backticks: ``
- Triple dashes: ---
- Angle brackets: <>
- XML tags: <tag> </tag>

In [ ]:
# "Stop children using smartphones until they are 13, says French report", The Guardian, 30th of April 2024 (https://www.theguardian.com/world/2024/apr/30/stop-children-using-smartphones-until-they-are-13-say-french-experts-in-report)
text = """
Children should not be allowed to use smartphones until they are 13 and should be banned from accessing \
conventional social media such as TikTok, Instagram and Snapchat until they are 18, according to a report \
by experts commissioned by Emmanuel Macron.

The French president had asked scientists and experts to suggest screen use guidelines for children with \
a view to France taking unprecedented steps on limiting their exposure. It was unclear how the government \
might now proceed after the report's publication. Macron said in January: “There might be bans, there might \
be restrictions.”

The hard-hitting report said children needed to be protected from the tech industry's profit-driven “strategy \
of capturing children's attention, using all forms of cognitive bias to shut children away on their screens, \
control them, re-engage them and monetise them”.
"""
prompt = f"""
Summarize the text delimited by triple backticks into a single sentence
```{text}```
"""

show_completion(prompt)

#### Instruct the model to generate a structured output

Examples
HMTL
JSON

In [ ]:
prompt = """
Provide a list of five movies with the corresponding director and genres.

Your answer should be a well formatted JSON following the keys: movie_id, title, director, genre.

Please just provide the JSON without any additional text or explanation. Also do not encase it in triple backticks.
"""

json_response = get_completion(prompt)

render_markdown(f"```json\n{json_response}\n```")

> [!Note]
> There is not guarantee that the schema of the answer is going to be always correct.

If the model provides a properly structured output, we can use it to parse the answer and use it in our application. If the model does not provide a properly structured output, we can use the answer as a text and parse it ourselves.

In [ ]:
parsed_response = json.loads(json_response)

movie_info_template = """
-----------------
MOVIE ID: {movie_id}
TITLE: {title}
DIRECTOR: {director}
GENRE: {genre}
-----------------
"""

for response in parsed_response:
    movie_info = movie_info_template.format(
        movie_id=response["movie_id"], title=response["title"], director=response["director"], genre=response["genre"]
    )
    print(movie_info)

#### Check whether the conditions are met

In [ ]:
text_1 = """
<text>To make Eggs Benedict, first start by poaching eggs in simmering water with a dash of vinegar to \
help set the whites. Next, while the eggs cook, toast English muffins until golden brown. Third, \
warm slices of Canadian bacon in a skillet. Once all the ingredients are ready, assemble the dish \
by placing the toasted muffin halves on a plate, topping each half with a slice of Canadian bacon,
followed by a poached egg. Lastly, drizzle hollandaise sauce over the eggs and garnish with chopped \
parsley or paprika for a classic touch. Serve immediately and enjoy the rich, indulgent flavors of Eggs Benedict!</text>
"""

prompt = f"""
You are given a text delimited by tags <text> </text>. If it contains a sequence of instructions, \
re-write those instructions in the following format:

Step 1) Place instruction 1 here
Step 2) Place instruction 2 here
Step 3) Place instruction 3 here
...
Step N) Place instruction N here

If the text does not contain a sequence of instructions, then answer 'No instructions provided'.

<text>{text_1}</text>
"""

show_completion(prompt)

Let's see what happens with other kinds of text.

In [ ]:
text_2 = """
I soon learned to know this flower better. On the little prince's planet the flowers had \
always been very simple. They had only one ring of petals; they took up no room at all; they \
were a trouble to nobody. One morning they would appear in the grass, and by night they would \
have faded peacefully away. But one day, from a seed blown from no one knew where, a new flower \
had come up; and the little prince had watched very closely over this small sprout which was not \
like any other small sprouts on his planet. It might, you see, have been a new kind of baobab.
"""

prompt = f"""
You are given a text delimited by tags <text> </text>. If it contains a sequence of instructions, \
re-write those instructions in the following format:

Step 1) Place instruction 1 here
Step 2) Place instruction 2 here
Step 3) Place instruction 3 here
...
Step N) Place instruction N here

If the text does not contain a sequence of instructions, then answer 'No instructions provided'. \
Please, follow the examples delimited by <example> tags.

<text>{text_2}</text>
<example>type our examples</example>
"""

show_completion(prompt)

#### Insert examples to show the model to solve the task (few-shot prompting)

In [ ]:
prompt = """
The word "sunsapphire" refers to the sun's warm glow and the brilliant blue hues of the summer sky at sunset.

To "glintle" means to emit a soft, subtle sparkle or shimmer, typically associated with delicate or understated \
beauty. An example of a sentence that uses the word "glintle" is:
"""

get_completion(prompt)

###  How to let the model simulate thinking process

- Sometimes the model makes reasoning errors, which leads to a wrong conclusion.
- Other times the task given to the model is too complex to solve, so the model may generate an incorrect answer.
- A way to mitigate these errors is to structure the prompt in a series or a chain of actions, so that the model is going to spend more computational power to solve the task.

There are some techniques for giving the model time to think. We will explore the following ones:

- Define the steps to complete a task
- Let the model find the solution on its own before rushing to a conclusion

#### Define the steps to complete a task

```
Step 1: ...
Step 2: ...
...
Step N: ...
```

In [ ]:
text = """
On a Friday morning, Petra and Oliver, siblings bound by a bond stronger than blood, embarked on \
their usual journey to school. As they strolled through the bustling streets, the crisp autumn air \
danced around them, carrying the promise of the weekend ahead. Petra, with her curly hair bouncing \
in rhythm with her steps, chattered excitedly about the upcoming school project. Oliver, the quieter \
of the two, listened intently, his mind already drifting to the adventures awaiting them. Together, \
they navigated the cityscape, their laughter harmonizing with the hum of the waking city, painting \
the dawn with hues of warmth and anticipation.
"""

prompt = f"""
Perform the following actions:
1. Summarize the text delimited by triple backticks in 1 sentence.
2. Translate the summary into Spanish.
3. List each person name in the summary.
4. Output a json object that contains the following keys: summary, num_names (encased in triple backticks)

Text:
```{text}```
"""
show_completion(prompt)

Let's ask the model to structure and format the answer

In [ ]:
prompt_2 = f"""
Perform the following actions:
1. Summarize the text delimited by triple backticks in 1 sentence.
2. Translate the summary into Spanish.
3. List each person name in the summary.
4. Output a json object that contains the following keys: spanish_summary, num_names.

Use the following format:
Text: <text to summarize>
Summary: <summary>
Translation: <summary translation>
Names: <list of names in summary>
Output JSON: <json with summary and num_names encased in triple backticks>

Text: ```{text}``
"""
show_completion(prompt_2)

### Let the model find the solution on its own before rushing to a conclusion

In [ ]:
prompt = """
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need help working out the financials.
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost me a flat $100k per year, and \
an additional $10 / square foot
What is the total cost for the first year of operations as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""
show_completion(prompt)

Note: The student solution is incorrect. Maintenance cost should be calculated as "100,000 + 10x" because it is $10 / square foot.

It is possible to make the model generate the correct answer by giving the chance to find the solution on its own first.

In [ ]:
prompt = """
Your task is to determine if the student's solution is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem including the final total.
- Then compare your solution to the student's solution and evaluate if the student's solution is correct or not.
Don't decide if the student's solution is correct until you have done the problem yourself.

Question:
```
I'm building a solar power installation and I need help working out the financials.
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost me a flat $100k per year, and an additional \
$10 / square foot
What is the total cost for the first year of operations as a function of the number of square feet.
```
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```
"""
show_completion(prompt)

Note: LLMs are not efficient for performing calculations, or reasoning on mathematical problems.

### Model limitations

**Hallucinations**
- The model can make statements that may sound true, but actually they are not true.
- This happens because the model "does not know where the boundaries of the knowledge learnt during training are".

Example:

In [ ]:
prompt = """
Provide a description about Coca-Cola, the newest soap dish of Fairy
"""

show_completion(prompt)

Strategy to mitigate hallucinations:
- Find relevant information about the question.
- Inject in the prompt the relevant information found.
- Ask the model answer the question using the relevant information.

## Time to practise on your own